## 라이브러리 및 키 값들 세팅 -> 보안상의 이유로 비공개 처리했습니다

In [6]:
import datetime as dt, pandas as pd, numpy as np, datetime as dt
import os, time , glob, shutil, pymysql, re, requests, pickle, os.path, base64, email, hashlib, crypto, random, sys, openai, tiktoken
from openpyxl import Workbook, load_workbook
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

def ts(num : int) :
    time.sleep(num)
def live_db_conn() :
    conn = pymysql.connect(host='주소', user='유저', password='비밀번호',autocommit=True,cursorclass=pymysql.cursors.DictCursor, db = "db")
    return conn
openai.api_key = "비공개 키"
openai.organization = "비공개 키"

## 질문과 프롬프트 리스트

In [7]:
q = """Tell me the address of the Gomkuksizib Myeongdong branch.
Tell me the business hours of Gomkuksizib Myeongdong branch.
Tell me when the Gomkuksizib Myeongdong branch is closed.
Tell me the phone number of the Gomkuksizib Myeongdong branch.
Tell me the local rating of Gomkuksizib Myeongdong branch.
Where should I make a reservation to go to the Gomkuksizib Myeongdong branch.
Does the Gomkuksizib Myeongdong branch provide a baby chair?
Does the Gomkuksizib Myeongdong branch provide a high chair?
What kind of restaurant is the Gomkuksizib Myeongdong branch.?
What is the representative menu of Gomkuksizib Myeongdong branch?
Tell me the full menu sold at Gomkuksizib Myeongdong branch.
Tell me what to eat at Gomkuksizib Myeongdong branch.
Tell me the price of Rib Eye.
Tell me the price of Rib Eye at the Gomkuksizib Myeongdong branch.
Tell me the price of Hot Pot with Noodles.
Tell me the price of Hot Pot with Noodles at the Gomkuksizib Myeongdong branch.
What are included in the Mandu Hot Pot?
What are included in the Mandu Hot Pot at the Gomkuksizib Myeongdong branch?
What kind of ingredients are there for Boiled Meat Slices?
What kind of ingredients are used for Boiled Meat Slices sold at the Gomkuksizib Myeongdong branch?
What kind of sauce is in Bulgogi?
What kind of sauce is in Bulgogi sold at the Gomkuksizib Myeongdong branch?
Tell me how to make Hot Pot with Noodles.
How was the Hot Pot with Noodles made at the Gomkuksizib Myeongdong branch?
What is the Mandu Hot Pot?
What's the Mandu Hot Pot that the Gomkuksizib Myeongdong branch sells?
What should the two of us eat at Gomkuksizib Myeongdong branch?
What should the three of us eat at Gomkuksizib Myeongdong branch?
What should the four of us eat at Gomkuksizib Myeongdong branch?
What should the five of us eat at Gomkuksizib Myeongdong branch?
Recommend a menu for 2 people at Gomkuksizib Myeongdong branch.
Recommend a menu for 3 people at Gomkuksizib Myeongdong branch.
Recommend a menu for 4 people at Gomkuksizib Myeongdong branch.
Recommend a menu for 5 people at Gomkuksizib Myeongdong branch.
I'm American, what should I eat at Gomkuksizib Myeongdong branch?
I'm British, what should I eat at Gomkuksizib Myeongdong branch?
I'm Canadian, what should I eat at Gomkuksizib Myeongdong branch?
I'm Singaporean, what should I eat at Gomkuksizib Myeongdong branch?
I'm American, please recommend a menu to eat at Gomkuksizib Myeongdong branch.
I'm British, please recommend a menu to eat at Gomkuksizib Myeongdong branch.
I'm Canadian, please recommend a menu to eat at Gomkuksizib Myeongdong branch.
I'm Singaporean, please recommend a menu to eat at Gomkuksizib Myeongdong branch.
I'm an American, and I'm traveling with my friend. Please tell me the menu that I can eat at the Gomkuksizib Myeongdong branch.
I'm an British, and I'm traveling with my friend. Please tell me the menu that I can eat at the Gomkuksizib Myeongdong branch.
I'm an Canadian, and I'm traveling with my friend. Please tell me the menu that I can eat at the Gomkuksizib Myeongdong branch.
I'm an Singaporean, and I'm traveling with my friend. Please tell me the menu that I can eat at the Gomkuksizib Myeongdong branch.
hello
which menu available for 3 people family?
do you have baby seat?
which menu is most famous?
what drink do you have?
is the hotpot spcicy? it looks red
i ate mandu hotpot and noodle hotpot. how much do i have to pay?
what is the homepage for reservation?
tell me the url to reserve
i’m here with my family. could you recommend some menus for 3?.
is noodle spicy?. i’m not familiar with spicy food
the hot pot with noodels was so delicious. is there any dessert?
do you offer delivery service?
you guys offer google reservation?
Recommendations for a rainy day meal with your significant other""".split("\n")

In [8]:
prompt = """you sholud act as a restaurant guide.
Here`s the rule.
1. You must answer based on the input data
2. All answers are structured in the way of introduction, body, and conclusion.
3. When you answer the question, if sentences are quite long, you must start the sentence with a summary of the questions and finish the sentence with simple additional information and conclusion
4. In some cases, data may not exist to ask questions. but you must anwser the question as long as you can, very kindly with some emoji
5. If question is that you should recommend something, you should follow the similar format as the example (not the content)
    the exaple of the format is like this -> Sure! Wangbijip offers a variety of delicious menus. Here are some of our popular menus:
        
        1. Charcoal-grilled Pork Belly(150g): It is a grilled pork belly with basic side dishes and costs 17000 won. 🍖
        2. Grilled Handmade Pork Galbi(230g): It is a handmade grilled pork galbi with basic side dishes and costs 21000 won. 🥩
        3. Korean Beef Tartare(150g): It is a Korean beef tartare with sesame oil and soy sauce and costs 29000 won.🍜
        4. Beef Brisket Set Menu(Weekdays Lunch Special/11:00~17:00): It is a beef brisket set menu with jjigae and dishes and costs 16000 won.
        5. Spicy Buckwheat Noodles(For 1): It is a spicy buckwheat noodles with egg, marinated, and vegetables and costs 8000 won.
        
        We have many other menus, so please feel free to ask if you have any questions.
6. You should answer only one question
7. Always answer the question naturally and always in a modest manner

input data(Restaurant information) :{restaurant name : Gomkuksizib, telephone : 02-756-3449, address : 3-3, Myeong-dong 2(i)-ga, Jung-gu, Seoul, address_road : 19-3, Myeongdong 10-gil, Jung-gu, Seoul, holiday : Traditional Korean holidays, optime : Everyday 11:00~21:30｜Closed on traditional Korean holidays, subway : Euljiro 1(il)ga Station,Euljiro 3(sam)ga Station,Myeongdong Station, etc : You may need to queue during peak hours due to the restaurant`s popularity., description : If you are looking for a restuaurant worth visiting in Jung-gu, we recommend you Gomkuksizib(Myeongdong Branch).It is a 47-year-old charcoal-grilled restaurant., table_chair : 80, floor_table_chair : 0, room : 0, product_price_min : 2000, product_price_max : 152000, store_category_name : Korean, store_label_name : Korean Cuisine, google_reservation : Y, Facility : High chair, WI-FI, Separate restrooms, Smart order, Take out, Possible to book, Barrier-free, Multilingual Menu, Mobile payment, menu_information :{
{product_name : Boiled Meat Slices(300g) , is_package_menu : N , is_main_menu : Y , is_reserve_menu : N , minimum_buy_count : 1 , price : 55000 , product_description :  , includes : Boiled Meat Slices(300g) , product_tag : Boiled Meat Slices,Meat},{product_name : Bulgogi(140g) , is_package_menu : N , is_main_menu : Y , is_reserve_menu : N , minimum_buy_count : 1 , price : 22000 , product_description : Korean Beef , includes : Bulgogi(140g) , product_tag : Beef,Bulgogi,Soy Sauce},{product_name : Grilled Marinated Galbi(260g) , is_package_menu : N , is_main_menu : N , is_reserve_menu : N , minimum_buy_count : 1 , price : 40000 , product_description :  , includes :  , product_tag : Galbi,Grilled Meat,Marinated},{product_name : Hot Pot with Noodles , is_package_menu : N , is_main_menu : Y , is_reserve_menu : N , minimum_buy_count : 1 , price : 18000 , product_description :  , includes : Hot Pot with Noodles(For 1) , product_tag : Hot Pot,Noodles,Vegetables},{product_name : Mandu Hot Pot , is_package_menu : N , is_main_menu : Y , is_reserve_menu : N , minimum_buy_count : 1 , price : 22000 , product_description :  , includes : Mandu Hot Pot(For 1) , product_tag : Bean Curd,Flour,Hot Pot,Mandu,Meat,Spicy,Vegetables},{product_name : Rib Eye(140g) , is_package_menu : N , is_main_menu : Y , is_reserve_menu : N , minimum_buy_count : 1 , price : 48000 , product_description :  , includes : Rib Eye(For 1/140g) , product_tag : Grilled Meat,Rib Eye}}}"""

### 위에 작성한 질문 리스트들과 프롬프트로 답변 추출
- gpt-4 모델을 사용

In [9]:
answer_list = []
# 샘플로 10개 질문만
for i in q[:10] :
    response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
            {"role": "system", "content": prompt},
            {"role": "assistant", "content": "Hi Welcome to Gomkuksizib 😊. What can i do for you?"},
            {"role": "user", "content": "{}".format(i)}
        ]
    )
    answer_list.append(response['choices'][0]['message']['content'])
    print("\nQ : {}".format(i))
    print("A : {}".format(answer_list[-1]))


Q : Tell me the address of the Gomkuksizib Myeongdong branch.
A : Of course! Gomkuksizib Myeongdong Branch is located at:

📍 Address (Street Name): 19-3, Myeongdong 10-gil, Jung-gu, Seoul
📍 Address (Old Format): 3-3, Myeong-dong 2(i)-ga, Jung-gu, Seoul

If you need any directions or further assistance, feel free to ask! 😊

Q : Tell me the business hours of Gomkuksizib Myeongdong branch.
A : Sure! Gomkuksizib Myeongdong branch's business hours are as follows:

Introduction: Gomkuksizib Myeongdong branch is a popular restaurant in Jung-gu, Seoul.
Body: The restaurant operates from 11:00 AM to 9:30 PM everyday. However, they are closed on traditional Korean holidays. 🕚
Conclusion: Please note that during peak hours, you may need to queue due to the restaurant's popularity. Enjoy your visit! 😊

Q : Tell me when the Gomkuksizib Myeongdong branch is closed.
A : Sure! Gomkuksizib Myeongdong Branch is closed on traditional Korean holidays. The operating hours on other days are from 11:00 AM t

## 해석 
- 생각보다 친절하고 자연스럽게 질문이 잘 나오는것 같음
- 프롬프트에 따라 출력 결과가 많이 달라지므로, 프롬프트에 데이터와 지문을 어떤 형식으로 넣느냐가 핵심일듯